In [ ]:
# CS-GY 6643 Computer Vision
# Project 1 : Canny Edge Detector

# Partner 1
# Name     : Yash Ramakant Bhalerao 
# Email    : yb2145@nyu.edu
# Net ID   : yb2145
# N Number : N12539240

import numpy as np
import math
import cv2

# Step 1 : Smooth the input image with the given 7*7 Gaussian mask
def gaussian_smoothing(image):

	# gaussian filter of size 7X7
  gaussian_mask =  [[1,1,2,2,2,1,1],
                    [1,2,2,4,2,2,1],
                    [2,2,4,8,4,2,2],
                    [2,4,8,16,8,4,2],
                    [1,1,2,2,2,1,1],
                    [1,2,2,4,2,2,1],
                    [2,2,4,8,4,2,2]]

  centerI, centerJ = len(gaussian_mask) // 2, len(gaussian_mask[0]) // 2
  i, j = centerI, centerJ

  height, width = image.shape
  smoothed_image = np.zeros((height, width))

  # apply convolution at each pixel
  while i < height - centerI:
    j = centerJ
    while j < width - centerJ:
      smoothed_image[i][j] = convolution(gaussian_mask, image, i, j, centerI, centerJ)

      # normalization step
      smoothed_image[i][j] = smoothed_image[i][j] / 140
      j = j + 1
    i = i + 1

  return smoothed_image

# convolution operation
def convolution(mask, image, i, j, centerI, centerJ):

	mask_i, mask_j = 0, 0
	i = i - centerI
	j = j - centerJ
	flag = j
	sum = 0

	while mask_i < len(mask):
		mask_j = 0
		j = flag
		while mask_j < len(mask[0]):
			sum = sum + (mask[mask_i][mask_j] * image[i][j])
			j = j + 1
			mask_j = mask_j + 1
		mask_i = mask_i + 1
		i = i + 1

	return sum

# Step 2 : Prewitt Operation 
def prewitt_operation(image):

  # prewitt operator x
  prewitt_x = [[-1,0,1],
              [-1,0,1],
              [-1,0,1]]

  # prewitt operator y
  prewitt_y = [[1,1,1],
              [0,0,0],
              [-1,-1,-1]]

  height, width = image.shape
  gradient_x = np.zeros((height, width))
  gradient_y = np.zeros((height, width))
  gradient_magnitude = np.zeros((height, width))
  gradient_angle = np.zeros((height, width))

  centerX, centerY = 4, 4
  i = centerX
  j = centerY

  # calculate gradient_x, gradient_y, gradient magnitude and gradient angle for each pixel location
  while i < height - centerX:
    j = centerY
    while j < width - centerY:

      # apply prewitt operator x
      gradient_x[i][j] = convolution(prewitt_x, image, i, j, len(prewitt_x) // 2, len(prewitt_x) // 2)
      
      # absolute value
      gradient_x[i][j] = abs(gradient_x[i][j])

      # normalization step
      gradient_x[i][j] = gradient_x[i][j] / 3

      # apply prewitt operator y
      gradient_y[i][j] = convolution(prewitt_y, image, i, j, len(prewitt_y) // 2, len(prewitt_y) // 2)
      
      # absolute value
      gradient_y[i][j] = abs(gradient_y[i][j])

      # normalization step
      gradient_y[i][j] = gradient_y[i][j] / 3

      # calculate gradient magnitude
      gradient_magnitude[i][j] = np.sqrt(gradient_x[i][j]**2  + gradient_y[i][j]**2)

      # normalization step
      gradient_magnitude[i][j] = gradient_magnitude[i][j] / np.sqrt(2)

      # calculate gradient angle
      if gradient_x[i][j] == 0:
        if gradient_y[i][j] > 0:
          gradient_angle[i][j] = 90
        else:
          gradient_angle[i][j] = -90
      else:
        gradient_angle[i][j] = math.degrees(math.atan((gradient_y[i][j] / gradient_x[i][j])))

      if gradient_angle[i][j] < 0:
        gradient_angle[i][j] = gradient_angle[i][j] + 360

      j = j + 1
    i = i + 1

  return gradient_x, gradient_y, gradient_magnitude, gradient_angle

# Step 3 : Non-Maxima Suppression
def non_maxima_suppression(gradient_magnitude, gradient_angle):
    
    nms_suppressed = gradient_magnitude
    height, width = nms_suppressed.shape

    # Looping through every pixel of the grayscale image
    for i_x in range(width):
      for i_y in range(height):

        grad_ang = gradient_angle[i_y, i_x]

        # sector 0
        if(grad_ang > 337.5 or grad_ang <= 22.5) or (grad_ang > 157.5 and grad_ang <= 202.5):
            neighbor_1_x, neighbor_1_y = i_x-1, i_y
            neighbor_2_x, neighbor_2_y = i_x + 1, i_y

        # sector 1
        elif(grad_ang > 22.5 and grad_ang <= 67.5) or (grad_ang > 202.5 and grad_ang <= 247.5):
            neighbor_1_x, neighbor_1_y = i_x-1, i_y-1
            neighbor_2_x, neighbor_2_y = i_x + 1, i_y + 1

        # sector 2
        elif(grad_ang > 67.5 and grad_ang <= 112.5) or (grad_ang > 247.5 and grad_ang <= 292.5):
            neighbor_1_x, neighbor_1_y = i_x, i_y-1
            neighbor_2_x, neighbor_2_y = i_x, i_y + 1

        #sector 3
        elif(grad_ang > 112.5 and grad_ang <= 157.5) or (grad_ang > 292.5 and grad_ang <= 337.5):
            neighbor_1_x, neighbor_1_y = i_x-1, i_y + 1
            neighbor_2_x, neighbor_2_y = i_x + 1, i_y-1

        # Compare with neighbors based on sector value
        if width > neighbor_1_x >= 0 and height > neighbor_1_y >= 0:
            if gradient_magnitude[i_y, i_x] < nms_suppressed[neighbor_1_y, neighbor_1_x]:
                nms_suppressed[i_y, i_x] = 0
                continue

        if width > neighbor_2_x >= 0 and height > neighbor_2_y >= 0:
            if nms_suppressed[i_y, i_x] < nms_suppressed[neighbor_2_y, neighbor_2_x]:
                nms_suppressed[i_y, i_x] = 0

    return nms_suppressed

# Step 4 : Thresholding using thresholds at 25th, 50th and 75th percentile
def percentileThreshold(image):

  imgList = []
  height, width = image.shape
  thresholded25 = np.zeros((height, width))
  thresholded50 = np.zeros((height, width))
  thresholded75 = np.zeros((height, width))

  # append all intensities excluding 0 to a list to calculate thresholds
  for i in range(height):
    for j in range(width):
      if image[i][j] > 0:
        imgList.append(image[i][j])

  # calculate threshold based on 25th, 50th and 75th percentile
  t25 = np.percentile(imgList, 25)
  t50 = np.percentile(imgList, 50)
  t75 = np.percentile(imgList, 75)

  # call threshold function with image and threshold parameter
  thresholded25 = simpleThresholding(image, t25)
  thresholded50 = simpleThresholding(image, t50)
  thresholded75 = simpleThresholding(image, t75)

  # return thresholded images at 25th, 50th and 75th percentile
  return thresholded25, thresholded50, thresholded75

# threshold input image based on threshold value
def simpleThresholding(image, threshold):

  height, width = image.shape
  thresholded_image = np.zeros((height, width))

  # perform thresholding by setting value to 0 if below threshold, and 255 otherwise
  for i in range(height):
    for j in range(width):
      if image[i][j] < threshold:
        thresholded_image[i][j] = 0
      else:
        thresholded_image[i][j] = 255

  return thresholded_image

def main():

  # path of the input image
  src = '/content/Test patterns.bmp'

  # read input image
  input_image = cv2.imread(src, 0)

  # Step 1 : Gaussian smoothing using a 7*7 Gaussian mask
  smoothed_image = gaussian_smoothing(input_image)
  cv2.imwrite("image_2_smoothed_image.bmp", smoothed_image)

  # Step 2 : Prewitt Operation
  gradient_x, gradient_y, gradient_magnitude, gradient_angle = prewitt_operation(smoothed_image)
  cv2.imwrite("image_2_gradient_x.bmp", gradient_x)
  cv2.imwrite("image_2_gradient_y.bmp", gradient_y)
  cv2.imwrite("image_2_gradient_magnitude.bmp", gradient_magnitude)
  cv2.imwrite("image_2_gradient_angle.bmp", gradient_angle)

  # Step 3 : Non-Maxima Suppression
  suppressed_image = non_maxima_suppression(gradient_magnitude, gradient_angle)
  cv2.imwrite("image_2_suppressed_image.bmp", suppressed_image)

  # Step 4 : Thresholding using thresholds at 25th, 50th and 75th percentile of suppressed image
  thresholded25, thresholded50, thresholded75 = percentileThreshold(suppressed_image)
  cv2.imwrite("image_2_thresholded25.bmp", thresholded25)
  cv2.imwrite("image_2_thresholded50.bmp", thresholded50)
  cv2.imwrite("image_2_thresholded75.bmp", thresholded75)

#invoke main function to call the steps of Canny Edge Detector algorithm
if __name__ == "__main__":
  main()